<a href="https://colab.research.google.com/github/JakeOh/202505_BD50/blob/main/lab_da/ml21_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM(Large Language Model, 거대 언어 모델)

*   시퀀스-시퀀스 작업(Sequence-to-Sequence)
    *   시퀀스 데이터를 입력받아서 시퀀스 데이터를 출력하는 작업
    *   자연어 처리(NLP, Natural Language Processing) 분야에서 요약, 번역 등의 작업 해당
    *   두 개의 (순환) 신경망을 연결한 인코더-디코더(encoder-decoder) 구조가 널리 사용
*   어텐션 메커니즘(Attention mechanism)
    *   인코더-디코더 구조에서 사용된 순환 신경망의 성능을 향상시키기 위해서 고안
    *   기존에는 인코더의 마지막 타임스텝에서 출력한 은닉 상태만을 사용해서 디코더가 새로운 텍스트를 생성
    *   어텐션 메커니즘은 모든 타임스텝에서 인코더가 출력한 은닉 상태를 디코더가 참조할 수 있도록 고안
    *   디코더가 새로운 토큰을 생성할 때 인코더가 처리한 토큰들 중에서 어떤 토큰에 주의(attention)를 기울일 지를 결정. (입력 토큰들마다 디코더가 중요도를 다르게 부여)
*   트랜스포머 모델(Transformer model)
    *   어텐션 메커니즘을 기반으로 하여 인코더-디코더 구조에서 순환층을 제거
    *   인코더에서 한 번에 하나의 토크씩 처리하지 않고 입력 테스트 전체를 한 번에 처리
    *   핵심 구성 요소
        *   멀티 헤드 어텐션(multi-head attention)
        *   층 정규화(layer normalization)
        *   잔차 연결(residual connection)
        *   피드 포워드 네트워크(feed-forward network)

<img src="https://pbs.twimg.com/media/GJg3QtMXwAAvUT4?format=jpg&name=large"
    alt="LLM 가계도" />

*   Encoder 기반 모델
    *   텍스트 (긍정/부정) 분류
    *   개체명 인식 - 텍스트에서 사람 이름, 지역 이름, 회사 이름 등의 고유 명사를 식별.
    *   BERT, RoBERTa, ...
*   Encoder-Decoder 기반 모델
    *   문서 요약, 번역, 질문-답변(질문과 문맥 텍스트가 주어졌을 때 문맥 안에서 답을 찾아서 생성)
    *   T5, BART, ...
*   Decoder 기반 모델
    *   텍스트 생성 - 챗봇, 질문 답변, 요약, 번역
    *   디코더:
        *   이전까지 생성한 텍스트를 입력받아 다음 토큰을 예측하는 방식
        *   인코더로부터의 입력이 없으면 디코더는 아무것도 생성할 수 없음.
        *   이전에 생성한 텍스트인 것처럼 어떤 텍스트를 입력해 주면 인코더 도움 없이 다음 토큰을 예측.
        *   프롬프트(prompt): 이전에 생성한 텍스트인 것처럼 전달하는 초기 텍스트.
    *   GPT-4, GPT-5, LLaMA, ...
    *   현재 가장 활발히 연구되고 있는 LLM 분야.

# HuggingFace에서 KoBART 모델 사용하기

In [1]:
import transformers  # HuggingFace의 transformers 기반 모델을 사용하기 위한 패키지

In [ ]:
# HuggingFace에서 오픈 소스로 공개된 모델을 다운로드해서 로컬에서 실행할 수 있도록 해줌.
pipe = transformers.pipeline(task="summarization",  # 문서 요약 작업
                             model="sshleifer/distilbart-cnn-12-6")
# device=0: GPU 사용, device=-1(기본값): CPU

In [ ]:
type(pipe)  # Pipeline 클래스 객체

In [11]:
sample_text = \
'''
During the 1905–06 English football season, New Brompton F.C. competed in the Southern League Division One. It was the 12th season in which the club competed in the Southern League and the 11th in Division One. The team began the season in poor form; they failed to score any goals in six of their first eight Southern League games. By the midpoint of the season, the team had won only three times and were close to the bottom of the league table. The team's form improved in the new year, with three wins in the first seven Southern League games of 1906, but they ended the season in similar fashion to how they had started it, failing to score in eight of the final nine league games. New Brompton finished the season in 17th place out of 18 teams in the division.

New Brompton also competed in the FA Cup, reaching the second round. The team played a total of 37 league and cup matches, winning 8, drawing 9 and losing 20. Bill Marriott was the club's top goalscorer, with four goals in the Southern League and one in the FA Cup; this figure was the lowest to date with which a player had finished a season as New Brompton's top scorer. Joe Walton made the most appearances, playing in 36 of the team's 37 competitive games. The highest attendance recorded at the club's home ground, Priestfield Road, was 5,500 for a game against Portsmouth on 27 January 1906.

The club's first match of the season, on 2 September, was away to Queens Park Rangers; Floyd, Phillips, Sheridan and Marriott all made their Southern League debuts for New Brompton.[13] Queens Park Rangers scored two goals in each half to win 4–0; the correspondent for The Daily News stated that the home team were "the better side at every point" and well deserved their large victory.[14] The first Southern League game of the season at New Brompton's ground, Priestfield Road, a week later against Bristol Rovers, resulted in a 3–0 win for the away team.[13][15] Phillips scored New Brompton's first Southern League goal of the season on 16 September and the team achieved their first victory, beating Northampton Town 2–0,[13][16] before losing again in their next game, a 4–0 defeat away to Portsmouth.[17] After a goalless draw with Swindon Town, New Brompton were 15th out of 18 teams in the Division One table at the end of September.[18]
'''

In [12]:
# Pipeline 객체를 함수처럼 호출할 수 있음.
result = pipe(sample_text)

In [13]:
print(result[0]['summary_text'])

 During the 1905–06 English football season, New Brompton F.C. competed in the Southern League Division One . The team played 37 league and cup matches, winning 8, drawing 9 and losing 20 . The highest attendance recorded at the club's home ground, Priestfield Road, was 5,500 for a game against Portsmouth in January 1906 .


In [15]:
sample_text_kor = \
'''
1896년 하계 올림픽(영어: 1896 Summer Olympics, Games of the I Olympiad, 그리스어: Θερινοί Ολυμπιακοί Αγώνες 1896)은 393년을 마지막으로 끝난 고대 올림픽 대회 이후 열린 첫 근대 올림픽 대회다. 1896년 4월 6일부터 4월 15일까지 그리스 아테네에서 개최되었다. 고대 그리스가 올림픽의 발상지여서 첫 근대 올림픽이 열리기에 적당한 장소였던 아테네는 1894년 6월 23일에 파리에서, 프랑스의 역사학자인 쿠베르탱이 주관한 올림픽 의회에서 만장일치로 개최지 자격을 얻었다. 또한 하계 올림픽이 진행되는 동안 국제 올림픽 위원회(IOC)가 조직되었다.

여러 어려움을 이겨낸 1회 올림픽은 성공적인 평가를 받았다. 당시의 국제 경기 중에서는 이 대회가 가장 많은 국제적인 참여를 이끌어냈다. 19세기 때 유일한 올림픽 경기장으로 쓰인 파나티나이코 경기장은 경기를 보려는 사람들이 몰려서 인산인해를 이루었다.[2] 개최국인 그리스에게 있어서 가장 절정이었던 부분은 마라톤 경기에서 자국 선수인 스피리돈 루이스가 승리의 영광을 차지한 것이었다. 이 대회에서는 카를 슈만이 레슬링과 체조에서 4개의 금메달을 따서 가장 많은 금메달을 획득했다. 가장 많이 메달을 딴 선수는 헤르만 바인게르트너로 6개의 메달을 땄다.

대회가 끝난 후 IOC는 이후의 올림픽을 계속 그리스에서 개최할 것인가를 놓고 피에르 드 쿠베르탱 남작과 그리스 임금인 요르요스 1세, 몇몇 미국 선수들 사이에서 갈등이 있었으나 1900년 대회가 이미 파리에서 열리기로 결정된 상태였고, 이후 올림픽은 세계를 순환하면서 개최하게 된다. 1906년 중간 올림픽을 제외하고는 그리스는 2004년 하계 올림픽을 개최할 때까지 108년간 올림픽을 개최하지 못했다.
'''

In [ ]:
# pipe(sample_text_kor)
# 해당 모델은 한글 텍스트로는 훈련이 되지 않아서 요약 작업을 실행할 수 없음.

In [ ]:
pipe_kor = transformers.pipeline(task='summarization',
                                 model='EbanLee/kobart-summary-v3')

In [18]:
pipe_kor(sample_text_kor)

[{'summary_text': '1896년 하계 올림픽은 고대 올림픽 대회 이후 열린 첫 근대 올림픽 대회로 아테네에서 개최되었다.'}]